In [36]:
import numpy as np
import threading
from threading import Thread
import pandas as pd
import re
import nltk
import spacy
import string
import queue
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity
from sklearn.decomposition import TruncatedSVD
from io import StringIO
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import time
from collections import Counter
from subprocess import call

In [37]:
df = pd.read_csv('./1 Billion Citation Dataset/1 Billion Citation Dataset, v1 (104).csv')
df = pd.DataFrame(df)


In [38]:
df1 = df[:250000]
df2 = df[250000:500000]
df3 = df[500000:750000]
df4 = df[750000:1000000]
df5 = df[1000000:1250000]
df6 = df[1250000:1500000]
df7 = df[1500000:1750000]
df8 = df[1750000:2000000]
df9 = df[2250000:2500000]



In [39]:
nltk.download('stopwords')
",".join(stopwords.words("english"))
STOPW = set(stopwords.words("english"),)
def remove_stopwords(text):
  return" ".join([word for word in str(text).split() if word not in STOPW])      

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Thon\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
PUNCT = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","" , PUNCT))


In [41]:
def remove_urls(text):
  url_pattern = re.compile(r"https?://\S+|www\.\S+")
  return url_pattern.sub(r"", text)

def remove_html(text):
  html_pattern = re.compile("<.*?>")
  return html_pattern.sub(r"", text)
  

In [42]:
def Preprocess(df):
    Df_preprocess = Df_preprocess.str.lower()
    Df_preprocess = df.loc[:,'citationStringAnnotated']
    Df_preprocess = Df_preprocess.astype(str)    
    Df_preprocess = Df_preprocess.apply(lambda text: remove_urls(text))
    Df_preprocess = Df_preprocess.apply(lambda text: remove_html(text))
    Df_preprocess = Df_preprocess.apply(lambda text: remove_punctuation(text))
    Df_preprocess = Df_preprocess.apply(lambda text: remove_stopwords(text))
    Df_preprocess = Df_preprocess.str.replace('\d+', '',regex=True)

    return Df_preprocess

In [43]:
class ThreadWithReturnValue(Thread):
    def __init__(self, group=None, target=None, name=None,
                 args=(), kwargs={}, Verbose=None):
        Thread.__init__(self, group, target, name, args, kwargs)
        self._return = None
    def run(self):
        print(type(self._target))
        if self._target is not None:
            self._return = self._target(*self._args,
                                                **self._kwargs)
    def join(self, *args):
        Thread.join(self, *args)
        return self._return

In [47]:


dfa = []
thread1 = ThreadWithReturnValue(target = Preprocess, args=(df1,))
thread2 = ThreadWithReturnValue(target = Preprocess, args=(df2,))
thread3 = ThreadWithReturnValue(target = Preprocess, args=(df3,))
thread4 = ThreadWithReturnValue(target = Preprocess, args=(df4,))
thread5 = ThreadWithReturnValue(target = Preprocess, args=(df5,))
thread6 = ThreadWithReturnValue(target = Preprocess, args=(df6,))
thread7 = ThreadWithReturnValue(target = Preprocess, args=(df7,))
thread8 = ThreadWithReturnValue(target = Preprocess, args=(df8,))
thread9 = ThreadWithReturnValue(target = Preprocess, args=(df9,))


thread1.start()
df1 = thread1.join()

thread2.start()
df2 = thread2.join()

thread3.start()
df3 = thread3.join()

thread4.start()
df4 = thread4.join()

thread5.start()
df5 = thread5.join()

thread6.start()
df6 = thread6.join()

thread7.start()
df7 = thread7.join()

thread8.start()
df8 = thread8.join()

thread9.start()
df9 = thread9.join()




Exception in thread Thread-51:
Traceback (most recent call last):
  File "C:\Users\Thon\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Thon\AppData\Local\Temp/ipykernel_5072/2792683848.py", line 9, in run
  File "C:\Users\Thon\AppData\Local\Temp/ipykernel_5072/3944587485.py", line 2, in Preprocess
UnboundLocalError: local variable 'Df_preprocess' referenced before assignment
Exception in thread Thread-52:
Traceback (most recent call last):
  File "C:\Users\Thon\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Thon\AppData\Local\Temp/ipykernel_5072/2792683848.py", line 9, in run
  File "C:\Users\Thon\AppData\Local\Temp/ipykernel_5072/3944587485.py", line 2, in Preprocess
UnboundLocalError: local variable 'Df_preprocess' referenced before assignment
Exception in thread Thread-53:
Traceback (most recent call last):
  File "C:\Users\Thon\AppData\Lo

<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>
<class 'function'>


In [45]:
frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9]
result = pd.concat(frames)
display(result)

ValueError: All objects passed were None

In [ ]:
def build_lsa(dataset, dim, queue ):
        tfidf_vec = TfidfVectorizer(use_idf=True, norm='l2')
        svd = TruncatedSVD(n_components=dim)
    
        transformed_x_train = tfidf_vec.fit_transform(dataset)
    
        print('TF-IDF output shape:', transformed_x_train.shape)
    
        data_svd = svd.fit_transform(transformed_x_train)
    
        print('LSA output shape:', data_svd.shape)
    
        explained_variance = svd.explained_variance_ratio_.sum()
        print("Sum of explained variance ratio: %d%%" % (int(explained_variance * 100)))

        queue.put((tfidf_vec, transformed_x_train,svd, data_svd))
         

In [ ]:
queue1 = queue.Queue()
thread_lsa1 = threading.Thread(target = build_lsa, args = (result,50, queue1,)) 
thread_lsa1.start()
thread_lsa1.join()

TF-IDF output shape: (2250000, 71348)
LSA output shape: (2250000, 50)
Sum of explained variance ratio: 7%


In [ ]:
tfidf_vec1, transformed_x_train1,svd1, data_svd = queue1.get()

In [ ]:
l=data_svd1[0]

for i,topic in enumerate(l):
  print("Topic ",i+1," : ",topic*100)

Topic  1  :  15.440031048218303
Topic  2  :  0.10667157762753068
Topic  3  :  -4.361529940017907
Topic  4  :  -4.053099619463622
Topic  5  :  0.6812022060068269
Topic  6  :  7.113591146782172
Topic  7  :  11.797941568586221
Topic  8  :  -0.8200502907784988
Topic  9  :  0.9055247833318384
Topic  10  :  4.9476519954279805
Topic  11  :  -4.305002042458647
Topic  12  :  -5.3718374309834855
Topic  13  :  -1.7971744944243817
Topic  14  :  -2.9426876564885984
Topic  15  :  4.93131330680054
Topic  16  :  -6.253496452932958
Topic  17  :  -0.7289337392705852
Topic  18  :  -6.98476560842544
Topic  19  :  -1.3976694526786098
Topic  20  :  0.5308839062030681
Topic  21  :  3.2898591298412514
Topic  22  :  -1.1528644142334825
Topic  23  :  -3.3244129370933533
Topic  24  :  -1.705503404234607
Topic  25  :  -1.1877536118178231
Topic  26  :  0.6785441424077461
Topic  27  :  3.638795429730262
Topic  28  :  -2.149976638601254
Topic  29  :  -0.2184038051726869
Topic  30  :  -3.0116228246817616
Topic  31  :

In [ ]:
vocab = tfidf_vec1.get_feature_names()
list1 = []
list2 = []
for i, comp in enumerate(svd1.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i+1)+": ")
    list1.insert(i,np.array(sorted_words)[:,1])
    list2.append(" ".join(np.array(sorted_words)[:,0]))
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 1: 
journal available pp online doi american international internet et anon 



C:\Users\Thon\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic 2: 
ieee conference international th journal systems proceedings system wang based 

Topic 3: 
de ieee conference international press springer la university pp th 

Topic 4: 
press university oxford pp cambridge ieee online springer conference anon 

Topic 5: 
springer berlin heidelberg pp der nature available und online die 

Topic 6: 
available online anon internet reviews choice ieee book supsup ppp 

Topic 7: 
et al elsevier bv wang research internet zhang li cancer 

Topic 8: 
der und die für des internet von verlag zeitschrift available 

Topic 9: 
society american et available al internet engineers mechanical springer berlin 

Topic 10: 
reviews online et choice al der pp american lt vol 

Topic 11: 
doi review society american reviews internet news choice book nature 

Topic 12: 
elsevier society pp bv vol engineers mechanical chemical physics engineering 

Topic 13: 
american elsevier medical association bv review pp vol jama news 

Topic 14: 
elsevier bv medicine anon i

In [ ]:
listf = pd.DataFrame(list1)
listf

,0,1,2,3,4,5,6,7,8,9
0,0.3710124228069012,0.3224342979309027,0.2538004352391002,0.20941448896507944,0.20520977828470663,0.16275167058771495,0.15463310336257283,0.15078769000302322,0.1361705093000548,0.13160790098406233
1,0.4463295910637001,0.29296977722410605,0.2625567515108779,0.122831592957576,0.12225803093630266,0.10931727995877495,0.10142010238621932,0.0678258057996202,0.062488451869057,0.05930092558984209
2,0.3662247533082121,0.33910820612684556,0.2283207435217273,0.1413390516680909,0.14050286110447605,0.13813541893268733,0.1299746765425351,0.11509494783660393,0.10273249472742674,0.10039440301948578
3,0.376413336704475,0.36052706317074096,0.23380490752583605,0.11883959179385611,0.1023840965402318,0.06768461583859027,0.06571358058203952,0.059383616051267636,0.0570335203546138,0.055538022945246686
4,0.460449887534407,0.32124280311543274,0.3208679142832785,0.16400602728220032,0.15011401306018546,0.13876246747907034,0.10508167795991868,0.08586859659283005,0.07296868153884527,0.06545016247812788
5,0.5047192112962698,0.46338262702590766,0.27442125395652023,0.20808443865090007,0.19433843476386414,0.17764586663611992,0.06556908570834039,0.043456935009134036,0.03211416842529822,0.02667780966293305
6,0.44405068793496355,0.43179894388594114,0.1574754595065902,0.13024703852003414,0.09857452616023232,0.09332665882831022,0.08232888415089734,0.07903347901424979,0.07538789916458234,0.06929547706399145
7,0.5294150879181962,0.3431511541286854,0.2808610877536289,0.19080992078289583,0.15234277632191617,0.14349778050422948,0.12303966024105886,0.11990223174173581,0.11968121262994261,0.11763319884422913
8,0.4346302584408154,0.27571803812958556,0.20551567731219472,0.20019887726785293,0.19386942955554148,0.17082533039293982,0.14089555350145383,0.1274678469647771,0.12580323366140878,0.11064766014172767
9,0.30758085571952903,0.29910409576380137,0.26130835695879645,0.25382374383508627,0.2534549635944141,0.15010636474788241,0.14206205782371012,0.11846166414745476,0.11265340995797651,0.11191893808101376


In [ ]:
list_name = list(list2)

simila = {}

for index in range(listf.shape[0]):

    similarities_1 = linear_kernel(list1[index].reshape(1, -1) ,list1).flatten()
    related_words = (-similarities_1).argsort()[:10]

    simila.update({list_name[index]:[list_name[i] for i in related_words]})

df_results = pd.DataFrame(simila)
df_results.reset_index(inplace=True)
df_results = df_results.T
df_results

,0,1,2,3,4,5,6,7,8,9
index,0,1,2,3,4,5,6,7,8,9
journal available pp online doi american international internet et anon,available online anon internet reviews choice ...,der und die für des internet von verlag zeitsc...,american elsevier medical association bv revie...,journal available pp online doi american inter...,elsevier society pp bv vol engineers mechanica...,springer berlin heidelberg pp der nature avail...,cheminform nature abstract nono lt anon intern...,society american et available al internet engi...,reviews online et choice al der pp american lt...,anon research nd et news al editorial medical ...
ieee conference international th journal systems proceedings system wang based,available online anon internet reviews choice ...,der und die für des internet von verlag zeitsc...,doi review society american reviews internet n...,springer berlin heidelberg pp der nature avail...,american elsevier medical association bv revie...,anon research nd et news al editorial medical ...,elsevier society pp bv vol engineers mechanica...,journal available pp online doi american inter...,society american et available al internet engi...,cheminform nature abstract nono lt anon intern...
de ieee conference international press springer la university pp th,available online anon internet reviews choice ...,der und die für des internet von verlag zeitsc...,american elsevier medical association bv revie...,springer berlin heidelberg pp der nature avail...,journal available pp online doi american inter...,elsevier society pp bv vol engineers mechanica...,cheminform nature abstract nono lt anon intern...,society american et available al internet engi...,anon research nd et news al editorial medical ...,doi review society american reviews internet n...
press university oxford pp cambridge ieee online springer conference anon,available online anon internet reviews choice ...,der und die für des internet von verlag zeitsc...,springer berlin heidelberg pp der nature avail...,american elsevier medical association bv revie...,doi review society american reviews internet n...,elsevier society pp bv vol engineers mechanica...,anon research nd et news al editorial medical ...,et al elsevier bv wang research internet zhang...,journal available pp online doi american inter...,cheminform nature abstract nono lt anon intern...
springer berlin heidelberg pp der nature available und online die,available online anon internet reviews choice ...,der und die für des internet von verlag zeitsc...,springer berlin heidelberg pp der nature avail...,american elsevier medical association bv revie...,doi review society american reviews internet n...,elsevier society pp bv vol engineers mechanica...,anon research nd et news al editorial medical ...,journal available pp online doi american inter...,society american et available al internet engi...,cheminform nature abstract nono lt anon intern...
available online anon internet reviews choice ieee book supsup ppp,available online anon internet reviews choice ...,der und die für des internet von verlag zeitsc...,springer berlin heidelberg pp der nature avail...,american elsevier medical association bv revie...,doi review society american reviews internet n...,elsevier society pp bv vol engineers mechanica...,anon research nd et news al editorial medical ...,et al elsevier bv wang research internet zhang...,journal available pp online doi american inter...,society american et available al internet engi...
et al elsevier bv wang research internet zhang li cancer,available online anon internet reviews choice ...,der und die für des internet von verlag zeitsc...,american elsevier medical association bv revie...,doi review society american reviews internet n...,springer berlin heidelberg pp der nature avail...,et al elsevier bv wang research internet zhang...,anon research nd et news al editorial medical ...,journal available pp online doi american inter...,elsevier society pp bv vol engineers mechanica...,society american et 